# Combine Sports Reference and Kenpom data into one dataframe

In [5]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')

# head_names = ['School', 'W-L%', 'SRS', 'SOS', 'TmPts', 'OppPts', 
#               'Pace', 'ORtg', 'FTr', '3PAr', 'TS%', 'TRB%', 'AST%', 
#               'STL%', 'BLK%', 'eFG%', 'TOV%', 'ORB%', 'FT/FGA']

kpHeadNames=['Rank','School','Conf','WL','AdjEM','AdjO','AdjD','AdjT','Luck','AdjEM','OppO','OppD','NCSOS_AdjEM','junk']

def num(s):
    try:
        return int(s)
    except ValueError:
        return float(s)


years = ['2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010']


In [58]:
kpData=pd.read_csv('data/kp/kp-2003.csv',names=kpHeadNames).drop('junk',axis=1)
teamNamesKP=kpData.School.values

newTeamList=[]
for t in teamNamesKP:
    newTeamList.append(t.lower().replace(" ","-").replace("st.","state"))
teamNamesKP=newTeamList

kpKeys=kpData.keys()
# kpKeys

In [59]:
srData=pd.read_csv('data/team/team_stats_2017.csv')
teamNamesBR=srData.School

Matchup all names for consistency

In [60]:
print(len(teamNamesBR))
print(len(teamNamesKP))

351
327


In [61]:
teamNameMismatch=[]
for teamName in teamNamesKP:
    if teamName not in set(list(teamNamesBR)):
        teamNameMismatch.append(teamName)
len(teamNameMismatch)

48

In [89]:
teamNameConversionKP2BR={
'smu': 'southern-methodist',
 "saint-mary's": "saint-mary's-(ca)",
 'tcu': 'texas-christian',
 'miami-fl': 'miami-(fl)',
 'vcu': 'virginia-commonwealth',
 'usc': 'southern-california',
 'unc-wilmington': 'north-carolina-wilmington',
 'california': 'university-of-california',
 'ucf': 'central-florida',
 'ut-arlington': 'texas-arlington',
 'byu': 'brigham-young',
 'state-bonaventure': 'st.-bonaventure',
 'loyola-chicago': 'loyola-(il)',
 "state-john's": "st.-john's-(ny)",
 'unc-asheville': 'north-carolina-asheville',
 'unc-greensboro': 'north-carolina-greensboro',
 'albany': 'albany-(ny)',
 'fort-wayne': 'ipfw',
 'penn': 'pennsylvania',
 'lsu': 'louisiana-state',
 'texas-a&m-corpus-chris': 'texas-a&m-corpus-christi',
 'uab': 'alabama-birmingham',
 'utep': 'texas-el-paso',
 'umbc': 'maryland-baltimore-county',
 "mount-state-mary's": "mount-st.-mary's",
 'umkc': 'missouri-kansas-city',
 'loyola-md': 'loyola-(md)',
 'usc-upstate': 'south-carolina-upstate',
 'little-rock': 'arkansas-little-rock',
 'unlv': 'nevada-las-vegas',
 'bowling-green': 'bowling-green-state',
 'utsa': 'texas-san-antonio',
 'liu-brooklyn': 'long-island-university',
 'state-francis-pa': 'saint-francis-(pa)',
 'fiu': 'florida-international',
 'miami-oh': 'miami-(oh)',
 'the-citadel': 'citadel',
 'umass-lowell': 'massachusetts-lowell',
 'ut-rio-grande-valley': 'texas-rio-grande-valley',
 'detroit': 'detroit-mercy',
 'prairie-view-a&m': 'prairie-view',
 'southern-miss': 'southern-mississippi',
 'grambling-state': 'grambling',
 'central-connecticut': 'central-connecticut-state',
 'state-francis-ny': 'st.-francis-(ny)',
 'troy-state': 'troy',
#  'southwest-missouri-state',
 'middle-tennessee-state': 'middle-tennessee',
 'southwest-texas-state': 'texas-state',
 'birmingham-southern': 'alabama-birmingham',
 'centenary': 'centenary-(la)',
#  'texas-pan-american',
#  'morris-brown'
}

In [90]:
teamNamesKPConverted=[]
for teamName in teamNamesKP:
    if teamName in teamNameConversionKP2BR.keys():
        teamNamesKPConverted.append(teamNameConversionKP2BR[teamName])
    else:
        teamNamesKPConverted.append(teamName)
len(teamNamesKPConverted)

327

Remaining number of mismatches

In [91]:
teamNameMismatch=[]
for teamName in teamNamesKPConverted:
    if teamName not in set(list(teamNamesBR)):
        teamNameMismatch.append(teamName)
len(teamNameMismatch)

4

In [92]:

teamNameMismatch

['southwest-missouri-state',
 'centenary-(la)',
 'texas-pan-american',
 'morris-brown']

In [93]:
tquery='california-davis'
for teamName in teamNamesBR:
    if tquery in teamName:
        print(tquery, ' is in')
        

# Combine dataframes

In [94]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt


In [98]:
for yr in np.arange(2017,1992,-1):
    # pull kenpom data
    print(yr)
    try:
        kpData=pd.read_csv('data/kp/kp-'+str(yr)+'.csv',names=kpHeadNames).drop('junk',axis=1)
    except:
        for key in kpKeys:
            try:
                kpData[key]=kpData[key]*np.nan
            except:
                pass
    teamNamesKP=kpData.School.values

    #basic formatting of school names
    newTeamList=[]
    for t in teamNamesKP:
        newTeamList.append(t.lower().replace(" ","-").replace("st.","state"))
    teamNamesKP=newTeamList

    # convert remaining names
    teamNamesKPConverted=[]
    for teamName in teamNamesKP:
        if teamName in teamNameConversionKP2BR.keys():
            teamNamesKPConverted.append(teamNameConversionKP2BR[teamName])
        else:
            teamNamesKPConverted.append(teamName)

    kpData.School=teamNamesKPConverted
    

    srData=pd.read_csv('data/team/team_stats_'+str(yr)+'.csv')
    teamNamesBR=srData.School

    if yr <= 2003:
        allData = srData
        for key in kpKeys:
            allData[key]=''
    else:
    
        allData=srData.merge(kpData,how='inner',on='School')
        allData=allData.rename(columns={'W%': 'wpct'})
    
    allData.to_csv("data/combined_team_stats_" + str(yr) + ".csv")
    
    
    
#     break
    # allData.sort_values(by='W-L%',ascending=False)
#     wpct=[]
#     for r in allData.itertuples():
#         w,l = r.WL.split('-')
#         wpct.append(float(w)/(float(w)+float(l)))
#     allData['wpct2']=wpct
#     #     print()
#     plt.figure()
#     plt.title(yr)
#     allData[['wpct','wpct2']]
#     plt.hist((allData['wpct']-allData['wpct2']));

2017
2016
2015
2014
2013
2012
2011
2010
2009
2008
2007
2006
2005
2004
2003
2002
2001
2000
1999
1998
1997
1996
1995
1994
1993
